<a href="https://colab.research.google.com/github/AmaniAlqarni/AI-Models/blob/main/Simple_Linear_Model_%2B_Fitting_the_Simple_Linear_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = (
    pd.read_csv('gdrive/MyDrive/Teaching/Fall2022/CS410/Colabs/Linear_models/mobility.csv')
    # filter out rows with NaN AUM values
    .query('not aum.isnull()', engine='python')
)
df

,cz,czname,stateabbrv,aum,rm,pop2000,intersects_msa,cs_race_bla,cs_race_theil_2000,cs00_seg_inc,...,mig_inflow,mig_outflow,cs_born_foreign,scap_ski90pcm,rel_tot,crime_violent,cs_fam_wkidsinglemom,cs_divorced,cs_married,incgrowth0010
0,100.0,Johnson City,TN,38.387500,0.37170,576081,1.0,0.020841,0.090384,0.034866,...,0.005640,0.004697,0.011784,-0.297858,0.514403,0.001410,0.189803,0.110173,0.600893,-0.002278
1,200.0,Morristown,TN,37.776752,0.35611,227816,1.0,0.019779,0.093153,0.026281,...,0.016206,0.014235,0.023055,-0.767355,0.543895,0.001844,0.185106,0.115958,0.613359,-0.002153
2,301.0,Middlesborough,TN,39.049250,0.37557,66708,0.0,0.014646,0.064250,0.024081,...,0.008050,0.011603,0.007078,-1.270251,0.667806,0.000854,0.211003,0.113451,0.590280,-0.003712
3,302.0,Knoxville,TN,37.841248,0.37253,727600,1.0,0.056365,0.209994,0.092104,...,0.016303,0.013554,0.019967,-0.221885,0.601953,0.001344,0.205602,0.114278,0.575150,-0.001997
4,401.0,Winston-Salem,NC,36.969250,0.38913,493180,1.0,0.173531,0.262388,0.071867,...,0.022101,0.018620,0.052948,-0.017796,0.487937,0.002658,0.220471,0.092351,0.585897,-0.014049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,39205.0,John Day,OR,49.717500,0.16578,7935,0.0,0.001008,0.001753,0.002285,...,0.003151,0.015375,0.015178,0.208387,0.331317,0.000252,0.194529,0.107960,0.627703,0.007669
737,39301.0,Friday Harbor,WA,39.102750,0.25523,14077,0.0,0.002415,0.009889,0.012479,...,0.024011,0.020885,0.059539,2.716138,0.171485,0.000426,0.218750,0.147890,0.603727,-0.010403
738,39302.0,Bellingham,WA,44.122753,0.27619,166814,1.0,0.006384,0.057326,0.046247,...,0.034008,0.027815,0.098455,0.062548,0.293710,0.001373,0.194839,0.098918,0.537829,-0.006947
739,39303.0,Port Angeles,WA,41.405502,0.28534,90478,0.0,0.006852,0.122457,0.025138,...,0.033909,0.021309,0.043290,0.475857,0.260207,0.001227,0.235148,0.123696,0.598164,0.000049


In [ ]:
import plotly.express as px
px.scatter(df, x='frac_traveltime_lt15', y='aum',
           width=350, height=250)

In [ ]:
def theta_1(x, y):
    r = x.corr(y)
    return r * y.std() / x.std()

In [ ]:
def theta_0(x, y):
    return y.mean() - theta_1(x, y) * x.mean()

In [ ]:
x = df['frac_traveltime_lt15']
y = df['aum']

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42,
)

print(f'Training set size: {len(x_train)} rows')
print(f'    Test set size: {len(x_test)} rows')

Training set size: 567 rows
    Test set size: 142 rows


In [ ]:
t1 = theta_1(x_train, y_train)
t0 = theta_0(x_train, y_train)
print(f'Model: f(x) = {t0:.2f} + {t1:.2f}x')

Model: f(x) = 33.15 + 24.27x


In [ ]:
import numpy as np
def mse(y, pred):
    return np.mean((y - pred) ** 2)

pred = t0 + t1 * x_test
print(f'Test set MSE: {mse(y_test, pred):.2f}')

Test set MSE: 17.47


In [ ]:
import plotly.graph_objects as go
fig = px.scatter(x=x_train, y=y_train,
                 width=450, height=250)

pred_x = np.array([x.min(), x.max()])
pred_y = t0 + t1 * pred_x

fig.add_trace(go.Scatter(x=pred_x, y=pred_y, name='Predictions', mode='lines'))
fig

We can see that the line has a positive slope, which captures the positive association between AUM adn the fraction of people with <=15 min commute.

In [ ]:
predicted = t0 + t1 * x_train
resid = pd.DataFrame({'predicted': predicted,
                      'residuals': y_train - predicted})
fig = px.scatter(resid, x='predicted', y='residuals',
                 width=350, height=250)

fig.add_hline(0, line_color='black', line_width=2, line_dash='dash', opacity=1)

Residual plots can show problems with using a linear model. A good fit will show a residual plot with a cloud of points around the y = 0 line with no clear pattern. When there is a clear pattern, we can usually conclude that a linear model wasn't a good choice for the data